In [1]:
def test_func(x):
    x = x + 1
    x = x * 2
    x = x ** 2
    x = x*x
    return x

In [2]:
%load_ext line_profiler
%lprun -f test_func test_func(1)

Timer unit: 1e-07 s

Total time: 3.3e-06 s
File: C:\Users\neelm\AppData\Local\Temp\ipykernel_11408\1388560847.py
Function: test_func at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def test_func(x):
     2         1          8.0      8.0     24.2      x = x + 1
     3         1          7.0      7.0     21.2      x = x * 2
     4         1          9.0      9.0     27.3      x = x ** 2
     5         1          5.0      5.0     15.2      x = x*x
     6         1          4.0      4.0     12.1      return x

In [14]:
%%timeit
X = jnp.array([1.0, 2.0, 3.0], dtype = jnp.float64)
z = jnp.array(X[2], dtype=int)
zp = jnp.array(X[2], dtype=int)
z==zp

529 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [2]:

from jax import config

config.update("jax_enable_x64", True)
import sys
from dataclasses import dataclass

import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_probability as tfp
from jax import grad, hessian, jit
from jaxtyping import Array, Float, install_import_hook
from matplotlib import rcParams
from tqdm.notebook import tqdm

sys.path.append("src")
from kernels.ArtificialKernel import ArtificialKernel
from kernels.DiagonalKernel import DiagonalKernel
from kernels.DivFreeKernel import DivFreeKernel
from utils.data_tools import generate_data, transform_data, regular_train_points, add_collocation_points
from utils.misc import rmse
from utils.plotting_tools import plot_data, plot_pred

with install_import_hook("gpjax", "beartype.beartype"):
    import gpjax as gpx

# Enable Float64 for more stable matrix inversions.
plt.style.use(
    "https://raw.githubusercontent.com/JaxGaussianProcesses/GPJax/main/docs/examples/gpjax.mplstyle"
)
colors = rcParams["axes.prop_cycle"].by_key()["color"]

import toml

params = toml.load("params.toml")["2D"]
a = params["a"]
noise = params["noise"]
N_train = params["N_train"]
N_test_1D = params["N_test_1D"]
N_c_list = params["N_c_list"]


n = 50 # number of training points
N_test_1D = 20 # number of test points in 1D
N_P = N_test_1D**2 # total number of test points

master_key = jr.key(102)

x,y,xtest,ytest = generate_data(master_key,a,n,N_test_1D,noise)
# x,y = regular_train_points(key = master_key, n=n, noise=noise)

# Plot the training and test data in a quiver plot
dataset_train, dataset_test = transform_data(x,y,xtest,ytest)

In [14]:
import numpy as np

# x1_test = jnp.linspace(0, 4, 20)
# x2_test = jnp.linspace(0, 4, 20)
x1_test = jnp.linspace(0, 4, 20)
x2_test = jnp.linspace(0, 4, 20)
x3_test = jnp.linspace(0, 4, 20)

X,Y,Z = np.meshgrid(x1_test, x2_test, x3_test)

In [4]:
xtest.shape

(400, 60)

In [9]:
jnp.dstack(jnp.meshgrid(x1_test, x2_test, x3_test)).shape

(20, 20, 60)

In [18]:
import jax.numpy as jnp

# Define the range and step size for each dimension
x = jnp.linspace(-1, 1, 2)
y = jnp.linspace(-1, 1, 2)
z = jnp.linspace(-1, 1, 2)

# Create the 3D grid using meshgrid
X, Y, Z = jnp.meshgrid(x, y, z, indexing='ij')

# Reshape the grids and stack them into a single (5^3, 3) array
xtest = jnp.stack([X, Y, Z], axis=-1).reshape(-1, 3)

print("3D Grid points (5^3 x 3):\n", grid)

In [19]:
def h(inp):
    x = inp[:, 0]
    y = inp[:, 1]
    z = inp[:, 2]

    r = jnp.sqrt(x**2 + y**2 + z**2)
    return jnp.where(r * (r + z) == 0, jnp.nan, x / (r * (r + z)))

In [21]:

h(xtest).shape

(8,)

In [22]:

    x = jnp.linspace(0, 4, 20)
    y = jnp.linspace(0, 4, 20)
    z = jnp.linspace(0, 4, 20)

    # Create the 3D grid using meshgrid
    X, Y, Z = jnp.meshgrid(x, y, z, indexing="ij")

    # Reshape the grids and stack them into a single (5^3, 3) array
    xtest = jnp.stack([X, Y, Z], axis=-1).reshape(-1, 3)

    # get the test output data
    ytest = h(xtest)

In [23]:
xtest.shape

(8000, 3)

In [25]:
ytest

Array([       nan, 0.        , 0.        , ..., 0.05816817, 0.05543085,
       0.05283122], dtype=float64)